In [1]:
import torch
import random
import numpy as np
import time
from params import PARAMS

def train_model(dataloaders, dataset_sizes, model, criterion, optimizer, num_epochs, random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)

    since = time.time()

    # For saving history
    train_loss = torch.zeros(num_epochs)
    val_loss = torch.zeros(num_epochs)
    train_acc = torch.zeros(num_epochs)
    val_acc = torch.zeros(num_epochs)

    for epoch in range(num_epochs):
        # print(f'Epoch {epoch}/{num_epochs-1}')
        # print('-' * 10)
        for phase in ['train', 'val']:
            # print('#' * len(phase))
            # print(phase)
            # print('#' * len(phase))
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0.0

            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(PARAMS['DEVICE'])
                labels = labels.to(PARAMS['DEVICE'])

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects / dataset_sizes[phase]

            # Save history
            if phase == 'train':
                train_acc[epoch] = epoch_acc
                train_loss[epoch] = epoch_loss
            elif phase == 'val':
                val_acc[epoch] = epoch_acc
                val_loss[epoch] = epoch_loss
            # print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

        # print()

    time_elapsed = time.time() - since
    # print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed%60:.0f}s')

    
    return model, {'train_loss': train_loss.tolist(), 'train_acc': train_acc.tolist(), 'val_loss': val_loss.tolist(), 'val_acc': val_acc.tolist(), 'train_time': time_elapsed}
    

In [2]:
from models import GCNAuto

In [3]:
from train import init_model_params
model = GCNAuto(in_features=PARAMS['SEQ_LEN'], n_nodes=PARAMS['N_CHANNELS'], num_classes=PARAMS['N_CLASSES'], hidden_sizes=PARAMS['FCN_HIDDEN_SIZES'])
model = model.to(PARAMS['DEVICE'])
model = init_model_params(model, random_seed=PARAMS['RANDOM_SEEDS'][0])

In [4]:
from utils import load_data, prepare_data
X_train, y_train, X_test, y_test, label_map = load_data('dataset/train/cross_subject_data_0_5_subjects.pickle')

In [5]:
print(X_train.shape)
print(X_test.shape)

(293612, 64)
(59040, 64)


In [6]:
X_train.shape[0] + X_test.shape[0]

352652

In [7]:
np.vstack([X_train, X_test]).shape

(352652, 64)

In [8]:
X_train, y_train = prepare_data(X_train, y_train, PARAMS['SEQ_LEN'])
X_test, y_test = prepare_data(X_test, y_test, PARAMS['SEQ_LEN'])